In [2]:

# importing tkinter for gui
import tkinter as tk
from tkinter import *
from datetime import date
#from datetime import datetime
import pyttsx3
import datetime
import speech_recognition as sr
import wikipedia
import os
import smtplib
import threading
import time
import cv2
from requests import get
import pywhatkit as kit
#import pyjokes
import requests
from bs4 import BeautifulSoup
from pywikihow import search_wikihow
import sys
import easyocr
from gtts import gTTS
from translate import Translator
from langdetect import detect
import threading
global queue
import ObjectR as ob
from queue import Queue


engine = pyttsx3.init('sapi5')
voices= engine.getProperty('voices') #getting details of current voice
engine.setProperty('voice', voices[0].id)


queue = Queue()




import FaceR as fc


def face_detection(takeCommand,T):
    
    def face_detection_start():
        fc.face_recog_start()

    def face_detection_stop():
        fc.face_recog_stop()

    c1 = threading.Thread(target=face_detection_start)
    c1.start()
    

    while(True):
        
        query3 = takeCommand(T).lower()

        if 'quit'in query3 or 'close'in query3:
            face_detection_stop()
            break
    time.sleep(1)

    c1.join()

        


def object_detection(takeCommand,T):
    
    def object_detection_start():
        ob.object_Run()

    def object_detection_stop():
        ob.object_stop()

    t1 = threading.Thread(target=object_detection_start)
    t1.start()

    while(True):

        query3 = takeCommand(T).lower()

        if 'quit'in query3 or 'close'in query3:
            ob.object_stop()
            break
    t1.join()
        



def translate_lang(lang,takeCommand,T):
    
    reader = easyocr.Reader([lang])
    camera = cv2.VideoCapture(0)
    
    close = 0
    def command(s):
        while True:
            query3 = takeCommand(T).lower()

            if 'click' in query3:   
                queue.enque(1)
                break
            elif 'quit'in query3 or 'close'in query3:
                queue.enque(2)
                break
                
    T1=threading.Thread(target=command,args=('s',))
    T1.start()
    while(True):
        
        _,image = camera.read()
        cv2.imshow('frame', image)
        response = queue.dequeue()
        if response==1:
            a = cv2.imwrite('test1.jpg', image)
            break
        elif response==2:
            close = 1
            break

        if cv2.waitKey(1) & 0xFF == ord('s'):
            a = cv2.imwrite('test1.jpg', image)  #not wprking
            break
        
    camera.release()
    cv2.destroyAllWindows()

    if close == 0:
        text_list = reader.readtext('test1.jpg',  detail=0)
        if len(text_list) == 0:
            return None
            #print("Soory, i am not able to recognize text")
        else:
            text_comb=' '.join(text_list)
            source_lang = detect(text_comb)
            translator= Translator(from_lang=source_lang,to_lang="english")
            translation = translator.translate(text_comb)
            return translation
    elif close == 1:
        return 'close'


def speak(T, audio):
    engine.say(audio) 
    print(audio)
    T.insert(tk.END, "\n\n\n"+audio)
    engine.runAndWait() #Without this command, speech will not be audible to us.
    T.delete("1.0", END)

def wishMe(T):
    hour = int(datetime.datetime.now().hour)
    tt= time.strftime("%I:%M %p")
    if hour>=0 and hour <12:
        speak(T,f"Good Morning, its {tt}")
    elif hour>=12 and hour<18:
        speak(T,f"Good Afternoon, its {tt}")
    else:
        speak(T,f"Good Evening, its {tt}")
    
    speak(T,"I am Jarvis sir.")
    speak(T,"Please tell me how may i help you.")

        
def sendEmail(to, content):
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.login('movieverse1234@gmail.com', 'Movieverse@1234')
    server.sendmail('movieverse1234@gmail.com', to, content)
    server.close()
    
    

def takeCommand(T):
    #It takes microphone input from the user and returns string output

    r = sr.Recognizer()
    with sr.Microphone() as source:
        T.insert(tk.END, "\n\n\nListening...")
        print("Listening...")
        #r.pause_threshold = 1
        try:
            audio = r.listen(source,timeout=5,phrase_time_limit=5)
        except Exception as e:
            pass
    try:
        print("Recognizing...")  
        T.delete("1.0", END)
        T.insert(tk.END, "\n\n\nRecognizing...")
        query = r.recognize_google(audio, language='en-in') #Using google for voice recognition.
        
        T.delete("1.0", END)
        print(f"User said: {query}")  #User query will be printed.
        T.insert(tk.END, f"You said: {query}")
        
    except Exception as e: 
        T.delete("1.0", END)
        #speak("Say that again please...") #Say that again will be printed in case of improper voice
        return "None" #None string will be returned
    return query 


if __name__=="__main__" :

    #now = datetime.now()
    today = date.today()

    #hour = int(datetime.datetime.now().hour)
    current_time= time.strftime("%I:%M %p")
    #current_time = now.strftime("%H:%M")
    d2 = today.strftime("%B %d, %Y")
    
    def startExecution(T):
        wishMe(T)
        while True:
            query = takeCommand(T).lower()

            if 'jarvis' in query:
                query = query.replace("jarvis","")
                if 'what can you do' in query:
                    speak(T, 'I can do the following for you:\n 1) Get information from wikipedia.\n 2) Get time.\n 3) Send email.\n 4) Tell you where we are.\n 5) Tell you how to do whatever you want.\n 6) Translate text.')
                elif 'wikipedia' in query:
                    try:
                        speak(T,'Searching Wikipedia...')
                        query = query.replace("wikipedia","")
                        results = wikipedia.summary(query, sentences=2)
                        speak(T,"According to wikipedia")
                        print(results)
                        speak(T,results)
                    except Exception as e:
                        print(e)
                        speak(T,"Sorry sir, I am not able to find this")  

                elif 'the time' in query:
                    hour = int(datetime.datetime.now().hour)
                    tt= time.strftime("%I:%M %p")   
                    speak(T,f"The time is {tt}")

                elif 'email to rishabh' in query:
                    try:
                        speak(T,"What should I say?")
                        content = takeCommand(T).lower()
                        to = "rishabhjain072001@gmail.com"    
                        sendEmail(to, content)
                        speak(T,"Email has been sent!")
                    except Exception as e:
                        print(e)
                        speak(T,"Sorry, I am not able to send this email")  


                elif 'where i am' in query or 'where we are' in query:
                    speak(T,"Wait sir, let me check")
                    try:
                        ipAdd = requests.get("https://api.ipify.org").text
                        print(ipAdd)
                        url = 'https://get.geojs.io/v1/ip/geo/'+ipAdd+'.json'
                        geo_requests = requests.get(url)
                        geo_data = geo_requests.json()
                        #state = geo_data['state']
                        city = geo_data['city']
                        country = geo_data['country']
                        speak(T,f'Sir i am not sure, but i think we are in {city} city of {country} country.')
                    except Exception as e:
                        speak(T,'Sorry sir, Due to network issue i am not able to find where we are.')
                        pass

                elif 'activate how to do mod' in query:
                    speak(T,"How to do mode is activated.")
                    speak(T,"Please tell me what you want to know")
                    while True:
                        how = takeCommand(T).lower()
                        try:
                            if "none" in how:
                                pass
                            elif "exit" in how or "close" in how or "no" in how:
                                speak(T,"Okay sir, how to do mode is closed.")
                                break
                            else:
                                max_results = 1
                                how_to = search_wikihow(how, max_results)
                                assert len(how_to) == 1
                                #how_to[0].print()
                                speak(T,how_to[0].summary)
                                speak(T,"That's it sir.")
                                speak(T,"Anything else do you want to know?")
                        except Exception as e:
                            speak(T,'Sorry sir, i am not able to find this.')


                elif 'translate' in query:
                    speak(T,"Which language do you want me to translate?")
                    query2 = takeCommand(T).lower()
                    languages = {'hindi':'hi','german':'de','spanish':'es','french':'fr','italian':'it','japanese':'ja','korean':'ko','latin':'la','thai':'th','chinese':'ch_sim'}

                    if query2 in languages:

                        speak(T,'Starting translation.')
                        speak(T,'Make sure you put the object in front of camera properly.')
                        speak(T,'So that i can read it.')
                        translation = translate_lang(languages[query2], takeCommand,T)
                        if translation is None:
                            speak(T,"Soory sir, i am not able to recognize text")
                        elif translation is 'close':
                            speak(T,"Closed!")
                        else:
                            speak(T,'Here is the translation')
                            speak(T,translation)
                    elif 'english' in query2:
                        speak(T,"I think converting english language to english, will not make sense")
                    else:
                        speak(T,"Sorry sir, i don't know that language.")

                #elif 'temperature' in query:
                elif 'object detection' in query:
                    speak(T,'Starting object detection.')
                    speak(T,'Make sure you put the object in front of camera properly,')
                    speak(T,'So that i can detect it.')
                    object_detection(takeCommand,T)
                    
                elif 'face recognition' in query:
                    speak(T,'Starting face recognition.')
                    speak(T,'Make sure you put the face in front of camera properly,')
                    speak(T,'So that i can recognize it.')
                    face_detection(takeCommand,T)
                    #print("ggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggggg")

                elif "hello" in query or "hey" in query:
                    speak(T,"Hello sir, may i help you with something.")

                elif "how are you" in query:
                    speak(T,"I am fine sir, what about you.")

                elif "also good" in query or "fine" in query:
                    speak(T,"That's great to hear from you.")

                elif "thank you" in query or "thanks" in query:
                    speak(T,"It's my pleasure sir.")

                elif "you can sleep" in query or "sleep now" in query or 'quit' in query:
                    speak(T,"Okay sir, i am going to sleep you can call me anytime")
                    break
                

    
    def main(T):
        
        while True:
            query = takeCommand(T).lower()

            if 'jarvis' in query:
                query = query.replace("jarvis","")
                if 'wake up' in query or "are you there" in query or "hello" in query:
                    startExecution(T)
    
    # creating window
    window = tk.Tk()
    window.attributes('-fullscreen', True)
    window.title("Smart Glasses")
    window.configure(bg='black')

    
    
    def timeUpdate():
        current_time = time.strftime("%I:%M %p")
        
        label2.config(text=current_time)
        label2.after(1000, timeUpdate)
        
    # creating text label to display on window screen
    label = tk.Label(window, text=d2,fg="white", bg="black", font=("helvetica", 30),width = 20)
    label.pack(anchor='ne',pady=1)
    label2 = tk.Label(window, text=current_time,fg="white", bg="black", font=("helvetica", 50),width = 10)
    label2.pack(anchor='ne')
    
    timeUpdate()
        
    T = tk.Text(window, fg="white", bg="black", height=100, width=50,font=("helvetica", 30), borderwidth = 0,wrap=tk.WORD)

    T.pack()
    t1 = threading.Thread(target=main, args=(T,))
    t1.start()
    #T.insert(tk.END, "\n\n\n\n\nJust a text Widget in two lines")
    window.mainloop()




# Textual month, day and year	



Listening...
Recognizing...
User said: pic of Jarvis
Listening...
Recognizing...
User said: wake up Jarvis
Good Morning, its 10:04 AM
I am Jarvis sir.
Please tell me how may i help you.
Listening...
Recognizing...
User said: service translate
Listening...
Recognizing...
User said: Jarvis translate
Which language do you want me to translate?
Listening...
Recognizing...
User said: German
Starting translation.
Make sure you put the object in front of camera properly.
So that i can read it.


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Listening...


Exception in thread Thread-21:
Traceback (most recent call last):
  File "C:\Users\Rishabh Jain\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Rishabh Jain\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4d7d3d626fa4>", line 357, in main
    startExecution(T)
  File "<ipython-input-2-4d7d3d626fa4>", line 304, in startExecution
    translation = translate_lang(languages[query2], takeCommand,T)
  File "<ipython-input-2-4d7d3d626fa4>", line 117, in translate_lang
    response = queue.dequeue()
AttributeError: 'Queue' object has no attribute 'dequeue'



Recognizing...
Listening...
Recognizing...
Listening...
Recognizing...
Listening...
Recognizing...


Exception in thread Thread-27:
Traceback (most recent call last):
  File "<ipython-input-2-4d7d3d626fa4>", line 192, in takeCommand
    T.delete("1.0", END)
  File "C:\Users\Rishabh Jain\Anaconda3\lib\tkinter\__init__.py", line 3139, in delete
    self.tk.call(self._w, 'delete', index1, index2)
RuntimeError: main thread is not in main loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Rishabh Jain\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Rishabh Jain\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4d7d3d626fa4>", line 102, in command
    query3 = takeCommand(T).lower()
  File "<ipython-input-2-4d7d3d626fa4>", line 201, in takeCommand
    T.delete("1.0", END)
  File "C:\Users\Rishabh Jain\Anaconda3\lib\tkinter\__init__.py", line 3139, in delete
    self.tk.call(self._w, 'delete', index1, index2)